## Langchain Agents
Agents use LLM to determine what actions to perform and in what order. Agents can use set of predefined tools to solve complex tasks.

In [1]:
!pip install arxiv

In [ ]:
 from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from dotenv import load_dotenv, find_dotenv
from langchain.agents import Tool
from langchain.agents import initialize_agent

load_dotenv(find_dotenv())

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
tool_names = ["arxiv", "llm-math"] #archive with scientific papers
tools = load_tools(tool_names, llm=llm)

tool_list = [
    Tool(
        name="Arxiv",
        func=tools[0].run,
        description="Search in scientific papers"
    ),
    Tool(
        name="Calculation tool",
        func=tools[1].run,
        description="Perform calculations"
    )
]

agent = initialize_agent(tool_list, llm, agent="zero-shot-react-description", verbose=True)
agent.invoke("What the name of most famous scientific paper about text transformers? Could you process calculations mentioned in this paper to check if they are correct?")

/tmp/ipykernel_21198/3998349903.py:26: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(tool_list, llm, agent="zero-shot-react-description", verbose=True)




> Entering new AgentExecutor chain...
To answer this question, I need to identify the most famous scientific paper about text transformers and then review the calculations mentioned in it. The most well-known paper in this field is likely "Attention is All You Need" by Vaswani et al., which introduced the Transformer model. I will first confirm this by searching for the paper.

Action: Arxiv
Action Input: "Attention is All You Need" by Vaswani et al.
Observation: No good Arxiv Result was found
Thought:It seems there was an issue retrieving the paper from Arxiv. However, I know that "Attention is All You Need" by Vaswani et al. is indeed the seminal paper that introduced the Transformer model, which is widely recognized in the field of natural language processing.

To proceed with verifying the calculations mentioned in the paper, I will need to access the paper's content, particularly the sections that involve mathematical formulations or calculations. Since I cannot retrieve the pap

In [9]:
from langchain_community.agent_toolkits.load_tools import get_all_tool_names
print(get_all_tool_names())

['sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-lens', 'google-serper', 'google-scholar', 'google-finance', 'google-trends', 'google-jobs', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'merriam-webster', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'stackexchange', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'read_file', 'reddit_search', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api', 'requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal']


### Custom Tool

In [21]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.agents import AgentType
import nltk

# download NLTK data packages
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

class CustomNERTool(BaseTool):
    name = "NER tagger"
    description = "named entity recognition tagger"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        tokens = nltk.word_tokenize(query)
        tagged_tokens = nltk.pos_tag(tokens)
        return nltk.ne_chunk(tagged_tokens)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("not suppored")

tools = [CustomNERTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Tag NER text: John Doe is a founder of ExampleCorp that was created in 1986 in Australia.") 

[nltk_data] Downloading package punkt to /home/michal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/michal/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/michal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to /home/michal/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.




> Entering new AgentExecutor chain...
I need to identify the named entities in the given text using the NER tagger.
Action: NER tagger
Action Input: John Doe is a founder of ExampleCOrp that was created in 1986 in Australia.
Observation: (S
  (PERSON John/NNP)
  (ORGANIZATION Doe/NNP)
  is/VBZ
  a/DT
  founder/NN
  of/IN
  (ORGANIZATION ExampleCOrp/NNP)
  that/WDT
  was/VBD
  created/VBN
  in/IN
  1986/CD
  in/IN
  (GPE Australia/NNP)
  ./.)
Thought:I now know the final answer.

Final Answer: The named entities in the text are:
- PERSON: John
- ORGANIZATION: Doe, ExampleCOrp
- GPE (Geopolitical Entity): Australia

> Finished chain.


'The named entities in the text are:\n- PERSON: John\n- ORGANIZATION: Doe, ExampleCOrp\n- GPE (Geopolitical Entity): Australia'

### Evaluating Agents
solving tasks with agents can be evaluated by Trajectory Evaluator, which checks if agent resoning is going right path  
see: https://js.langchain.com/v0.1/docs/guides/evaluation/trajectory/trajectory_eval/